## PREPROCESS AND PREPARE CSV


In [ ]:
!pip install pandas datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

dataset = load_dataset("squad_v2", split="train")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
import random

all_paragraphs = list(set([d['context'] for d in dataset]))

triplets = []
#question,positive,negative
for item in tqdm(dataset):
    question = item['question']
    context = item['context']
    answers = item['answers']['text']
    is_impossible = (len(answers) == 0)

    if not is_impossible:
        positive_context = context

        negative_contexts = random.sample(
            [c for c in all_paragraphs if c != context],
            k=2
        )

        triplets.append({
            "question": question,
            "positive_context": positive_context,
            "negative_context_1": negative_contexts[0],
            "negative_context_2": negative_contexts[1]
        })


df = pd.DataFrame(triplets)
df.to_csv("retriever_triplets.csv", index=False)
print(f"Triplet dataset saved: {len(df)} examples")


100%|██████████| 130319/130319 [02:35<00:00, 836.98it/s]


Triplet dataset saved: 86821 examples


## Fine-Tune a Dense Retriever (Bi-Encoder) with Sentence Transformers

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import pandas as pd
from sentence_transformers import InputExample
from sentence_transformers import SentenceTransformer, losses, models
from torch.utils.data import DataLoader
from tqdm import tqdm


df = pd.read_csv("retriever_triplets.csv")

train_examples = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    train_examples.append(
        InputExample(
            texts=[row['question'], row['positive_context'], row['negative_context_1']]
        )
    )


100%|██████████| 86821/86821 [00:04<00:00, 19131.41it/s]


In [ ]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'

model = SentenceTransformer(model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from sentence_transformers import losses

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model)


In [ ]:
import logging

logging.basicConfig(
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO
)
logger = logging.getLogger(__name__)

logger.info("Starting retriever fine-tuning...")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    output_path='fine-tuned-retriever',
    show_progress_bar=True
)

logger.info(" Training complete! Model saved at 'fine-tuned-retriever'")


Step,Training Loss
500,0.017000
1000,0.019000
1500,0.072500
2000,0.077100
2500,0.073800
3000,0.071100
3500,0.066800
4000,0.066500
4500,0.065400
5000,0.071600


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('fine-tuned-retriever')

corpus = [
    "Paris is the capital of France.",
    "The moon is Earth's only natural satellite.",
    "Python is a popular programming language.",
    "The Great Wall of China is visible from space.",
    "Mona Lisa was painted by Leonardo da Vinci."
]

corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

query = "Who painted the Mona Lisa?"

query_embedding = model.encode(query, convert_to_tensor=True)

hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)

print("\nTop results:")
for hit in hits[0]:
    print(f"> Score: {hit['score']:.4f} \t Text: {corpus[hit['corpus_id']]}")



Top results:
> Score: 0.8933 	 Text: Mona Lisa was painted by Leonardo da Vinci.
> Score: 0.1876 	 Text: The Great Wall of China is visible from space.
> Score: 0.1402 	 Text: The moon is Earth's only natural satellite.


In [ ]:
!zip -r fine-tuned-retriever.zip fine-tuned-retriever

  adding: fine-tuned-retriever/ (stored 0%)
  adding: fine-tuned-retriever/config.json (deflated 48%)
  adding: fine-tuned-retriever/vocab.txt (deflated 53%)
  adding: fine-tuned-retriever/2_Normalize/ (stored 0%)
  adding: fine-tuned-retriever/README.md (deflated 65%)
  adding: fine-tuned-retriever/modules.json (deflated 62%)
  adding: fine-tuned-retriever/special_tokens_map.json (deflated 80%)
  adding: fine-tuned-retriever/tokenizer_config.json (deflated 73%)
  adding: fine-tuned-retriever/config_sentence_transformers.json (deflated 34%)
  adding: fine-tuned-retriever/1_Pooling/ (stored 0%)
  adding: fine-tuned-retriever/1_Pooling/config.json (deflated 57%)
  adding: fine-tuned-retriever/model.safetensors (deflated 8%)
  adding: fine-tuned-retriever/sentence_bert_config.json (deflated 4%)
  adding: fine-tuned-retriever/tokenizer.json (deflated 71%)


api key:6d4b953ea7beba28c09f18af7bc6f82c8794cd4f

In [ ]:
from google.colab import files
files.download('fine-tuned-retriever.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>